In [1]:
import tensorflow as tf
import numpy as np
from ptr_decoder import pointer_decoder
from pointer_network import PointerNetwork
from data_generator import *
%load_ext autoreload
%autoreload 2

In [2]:
input_dim = 1
batch_size = 1
seq_len = 8
hidden_dim = 100
lr = 0.01
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
pointer_net = PointerNetwork(hidden_dim, lr, 5, seq_len, batch_size=batch_size, input_dim=1)

Building graph
Building optimizer
Finished


In [4]:
opt = pointer_net.opt
loss = pointer_net.losses
encoder_inps = pointer_net.encoder_inps
decoder_inps = pointer_net.decoder_inps
targets = pointer_net.targets
prediction = pointer_net.prediction
validation = pointer_net.validation
sess.run(tf.initialize_all_variables())

In [12]:
for i in range(0, 1000):
    _encoder_inps, _targets, _decoder_inps = sorting_generator(seq_len, batch_size)
    _loss, _, _v = sess.run([loss, opt, prediction], feed_dict={encoder_inps:_encoder_inps, decoder_inps:_decoder_inps, 
                                               targets:_targets})
    if i%100 == 0:    
        _v = np.argmax(_v, axis=2).reshape(1, 8)
        correct = np.where(_targets == _v)[0].shape[0]/float(batch_size*seq_len)
        print np.all(_targets == _v, axis=1)
        print _targets
        print _v
        #print "Encoder", _encoder_inps.reshape(8)
        print correct
        
        print "Loss", _loss

[False]
[[4 1 0 3 6 2 5 7]]
[[4 1 0 3 6 2 5 5]]
0.875
Loss 2.41682
[False]
[[1 2 0 5 4 7 3 6]]
[[1 2 0 5 4 4 3 6]]
0.875
Loss 2.88573
[False]
[[5 0 1 2 6 4 3 7]]
[[0 0 1 2 6 4 3 7]]
0.875
Loss 3.24919
[ True]
[[6 5 2 4 7 3 0 1]]
[[6 5 2 4 7 3 0 1]]
1.0
Loss 0.798699
[False]
[[2 5 1 0 7 4 6 3]]
[[2 5 1 7 0 4 6 3]]
0.75
Loss 4.59587
[ True]
[[7 3 6 4 0 2 5 1]]
[[7 3 6 4 0 2 5 1]]
1.0
Loss 0.979079
[False]
[[3 1 6 7 4 5 0 2]]
[[3 1 6 7 7 5 0 2]]
0.875
Loss 3.62393
[ True]
[[1 5 7 0 3 2 6 4]]
[[1 5 7 0 3 2 6 4]]
1.0
Loss 1.15635
[False]
[[5 3 7 2 4 1 0 6]]
[[5 7 7 2 4 1 0 6]]
0.875
Loss 1.73718
[False]
[[7 6 1 3 4 5 2 0]]
[[7 6 1 3 4 2 5 0]]
0.75
Loss 3.16212


In [12]:
_targets

array([[7, 0, 6, 4, 5, 3, 2, 1],
       [0, 4, 5, 2, 6, 7, 3, 1],
       [0, 6, 4, 2, 7, 3, 1, 5],
       [5, 1, 6, 3, 7, 4, 2, 0],
       [3, 1, 2, 0, 6, 4, 5, 7],
       [7, 4, 3, 5, 6, 0, 2, 1],
       [4, 3, 2, 1, 5, 6, 0, 7],
       [2, 5, 0, 6, 3, 7, 1, 4],
       [0, 5, 7, 1, 4, 3, 6, 2],
       [4, 3, 7, 2, 0, 1, 5, 6],
       [4, 2, 3, 5, 6, 0, 7, 1],
       [1, 6, 7, 5, 2, 4, 3, 0],
       [3, 2, 7, 0, 4, 1, 6, 5],
       [6, 2, 5, 1, 7, 3, 0, 4],
       [2, 6, 3, 7, 1, 0, 4, 5],
       [4, 5, 7, 2, 6, 1, 0, 3],
       [3, 7, 0, 1, 4, 6, 5, 2],
       [4, 5, 0, 7, 1, 6, 3, 2],
       [7, 5, 2, 4, 0, 6, 3, 1],
       [6, 7, 1, 2, 3, 4, 5, 0],
       [7, 4, 0, 5, 3, 6, 2, 1],
       [6, 1, 5, 7, 3, 4, 0, 2],
       [0, 7, 6, 1, 4, 2, 5, 3],
       [3, 5, 7, 2, 1, 0, 6, 4],
       [2, 3, 6, 0, 4, 7, 5, 1],
       [5, 4, 3, 7, 2, 1, 0, 6],
       [1, 7, 0, 4, 3, 2, 6, 5],
       [1, 0, 5, 4, 6, 3, 2, 7],
       [4, 6, 7, 3, 1, 5, 2, 0],
       [4, 7, 6, 1, 5, 2, 0, 3],
       [5,

In [12]:
np.where(_targets == np.argmax(_v, axis=2).reshape(1, 5))

(array([0, 0, 0]), array([0, 3, 4]))